In [2]:
import warnings
warnings.filterwarnings('ignore') # uncomment this line to suppress warnings

import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

# Point 1

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Tracce/exam2.csv',names=['a1','a2','a3','a4','a5'])
df.head(5)

,a1,a2,a3,a4,a5
0,5.1,3.5,a,NaN,0
1,4.9,3.0,a,NaN,0
2,NaN,3.2,a,NaN,0
3,4.6,NaN,a,0.2,0
4,5.0,3.6,a,0.2,0


In [4]:
df.describe()

,a1,a2,a4,a5
count,141.000000,140.000000,137.000000,150.000000
mean,5.897872,3.036429,1.290511,1.000000
std,0.820232,0.437654,0.733934,0.819232
min,4.300000,2.000000,0.100000,0.000000
25%,5.200000,2.800000,0.400000,0.000000
50%,5.800000,3.000000,1.400000,1.000000
75%,6.400000,3.300000,1.800000,2.000000
max,7.900000,4.400000,2.500000,2.000000


In [5]:
print('number of rows:',df.shape[0])
print('number of columns:',df.shape[1])

number of rows: 150
number of columns: 5


# 2

In [6]:
print('number of rows with null values')
df.shape[0] - df.dropna().shape[0]

number of rows with null values


28

In [7]:
df1 = df.dropna().copy()
df1.shape

(122, 5)

# 3

In [8]:
print(df1['a3'].unique())

['a' 'b' 'c' 'd']


In [9]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
new_df = pd.DataFrame(enc.fit_transform(df1), columns = df1.columns)
print(new_df['a3'].unique())

[0. 1. 2. 3.]


# 4

In [10]:
X = new_df.iloc[:, 0:-1]
y = new_df.iloc[:, -1]
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,random_state = 1,train_size=0.7)
print(Xtrain.shape,Xtest.shape)

(85, 4) (37, 4)


# 5-6-7-8-9


In [11]:
model_lbls = [
              'dt', 
              'svc'
            ]

# Set the parameters by cross-validation
tuned_param_dt = [{'max_depth': [*range(1,20)]}]
tuned_param_svc = [{'kernel': ['rbf'], 
                    'gamma': [1e-3, 1e-4],
                    'C': [1, 10, 100, 1000],
                    },
                    {'kernel': ['linear'],
                     'C': [1, 10, 100, 1000],                     
                    },
                   ]

models = {
    'dt': {'name': 'Decision Tree       ',
           'estimator': DecisionTreeClassifier(), 
           'param': tuned_param_dt,
          },
    'svc':{'name': 'Support Vector      ',
           'estimator': SVC(), 
           'param': tuned_param_svc
          }
}

In [16]:
def print_results(model):
    print("Best parameters set found on train set:")
    print()
    # if best is linear there is no gamma parameter
    print(model.best_params_)
    print()
    
    print()
    print("Detailed classification report for the best parameter set:")
    print()
    print("The model is trained on the full train set.")
    print("The scores are computed on the full test set.")
    print()
    y_true, y_pred = ytest, model.predict(Xtest)
    '''
    print(classification_report(y_true, y_pred))
    print()
    '''
    print('accuracy score')
    print(accuracy_score(y_true, y_pred))
    print("confusion matrix")
    print(confusion_matrix(y_true, y_pred,normalize = 'true'))


In [17]:
results_short={}
for m in model_lbls:
        print("Trying model",m)
        clf = GridSearchCV(models[m]['estimator'],models[m]['param'], cv=5,
                           scoring='accuracy', 
                           return_train_score = False,
                           n_jobs = 2, # this allows using multi-cores
                           )

        clf.fit(Xtrain, ytrain)
        print_results(clf)
        results_short[m] = clf.best_score_

Trying model dt
Best parameters set found on train set:

{'max_depth': 3}


Detailed classification report for the best parameter set:

The model is trained on the full train set.
The scores are computed on the full test set.

accuracy score
0.918918918918919
confusion matrix
[[1.         0.         0.        ]
 [0.         0.875      0.125     ]
 [0.         0.05263158 0.94736842]]
Trying model svc
Best parameters set found on train set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}


Detailed classification report for the best parameter set:

The model is trained on the full train set.
The scores are computed on the full test set.

accuracy score
0.9459459459459459
confusion matrix
[[1.         0.         0.        ]
 [0.         0.9375     0.0625    ]
 [0.         0.05263158 0.94736842]]
